Reference:
- Tweepy Doc: https://docs.tweepy.org/en/latest/
- Cursor Tutorial: https://docs.tweepy.org/en/latest/cursor_tutorial.html?highlight=cursor#cursor-tutorial
- User Timeline Tutorial: https://docs.tweepy.org/en/latest/api.html#API.user_timeline
- API setting: https://www.youtube.com/watch?v=dvAurfBB6Jk
- Functions: https://py.plainenglish.io/scraping-tweets-with-tweepy-python-59413046e788
- Scraping codes: https://towardsdatascience.com/how-to-scrape-tweets-from-twitter-59287e20f0f1


In [25]:
from tweepy import OAuthHandler
import tweepy
import webbrowser
import json
import pandas as pd
import csv
import re
import string
# import preprocessor as p
import os
import time

In [2]:
# for privacy issue, API key is in json file (which is in .gitignore)
with open('Twitter_API_info.json') as f:
    api_info = json.load(f)
    f.close()

In [3]:
access_token = api_info['access_token']
access_token_secret = api_info['access_token_secret']
API_key = api_info['API_key']
API_key_secret = api_info['API_key_secret']

In [4]:
callback_uri = 'oob' # https://cfe.sh/twitter/callback

In [8]:
auth = tweepy.OAuthHandler(API_key, API_key_secret, callback_uri)
redirect_url = auth.get_authorization_url()
print(redirect_url)

https://api.twitter.com/oauth/authorize?oauth_token=k9DXOgAAAAABCxGzAAABd8zQmn8


In [9]:
webbrowser.open(redirect_url)

True

In [10]:
user_pin_input = input("What's the pin value? ")

What's the pin value? 0562047


In [11]:
user_pin_input

'0562047'

In [12]:
auth.get_access_token(user_pin_input)

('1065561575598231553-GV8Q9reqL24p0d84nIdcvB85IuyYj5',
 'A514L7kjPoAHC4I2fPWt8RY8iml1mHhy0AJZDNrE2wHXh')

In [13]:
api = tweepy.API(auth, wait_on_rate_limit=True)

In [14]:
me = api.me()
print(me.screen_name)

JinfeiZhu


In [16]:
me.name

'Jinfei Zhu'

In [28]:
def scraptweets(search_words, date_since, numTweets, numRuns):
    
    # Define a for-loop to generate tweets at regular intervals
    # We cannot make large API call in one go. Hence, let's try T times
    
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags']
                                )
    program_start = time.time()
    for i in range(0, numRuns):
        # We will time how long it takes to scrape tweets for each run:
        start_run = time.time()
        
        # Collect tweets using the Cursor object
        # .Cursor() returns an object that you can iterate or loop over to access the data collected.
        # Each item in the iterator has various attributes that you can access to get information about each tweet
        tweets = tweepy.Cursor(api.search, q=search_words, lang="en", since=date_since, tweet_mode='extended').items(numTweets)# Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]# Obtain the following info (methods to call them out):
        # user.screen_name - twitter handle
        # user.description - description of account
        # user.location - where is he tweeting from
        # user.friends_count - no. of other users that user is following (following)
        # user.followers_count - no. of other users who are following this user (followers)
        # user.statuses_count - total tweets by user
        # user.created_at - when the user account was created
        # created_at - when the tweet was created
        # retweet_count - no. of retweets
        # (deprecated) user.favourites_count - probably total no. of tweets that is favourited by user
        # retweeted_status.full_text - full text of the tweet
        # tweet.entities['hashtags'] - hashtags in the tweet# Begin scraping the tweets individually:
        noTweets = 0
        for tweet in tweet_list:# Pull the values
            username = tweet.user.screen_name
            acctdesc = tweet.user.description
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            usercreatedts = tweet.user.created_at
            tweetcreatedts = tweet.created_at
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']
            try:
                text = tweet.retweeted_status.full_text
            except AttributeError:  # Not a Retweet
                text = tweet.full_text# Add the 11 variables to the empty list - ith_tweet:
            ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, text, hashtags]# Append to dataframe - db_tweets
            db_tweets.loc[len(db_tweets)] = ith_tweet# increase counter - noTweets  
            noTweets += 1
        
        # Run ended:
        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)
        
        print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
        print('time take for {} run to complete is {} mins'.format(i+1, duration_run))
        
        time.sleep(920) #15 minute sleep time# Once all runs have completed, save them to a single csv file:
    from datetime import datetime
    
    # Obtain timestamp in a readable format
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')# Define working path and filename
    path = os.getcwd()
    filename = path + '/data/' + to_csv_timestamp + '_sahkprotests_tweets.csv'# Store dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index = False)
    
    program_end = time.time()
    print('Scraping has completed!')
    print('Total time taken to scrap is {} minutes.'.format(round(program_end - program_start)/60, 2))

In [ ]:
# Initialise these variables:
search_words = "COVID OR covid"
date_since = "2020-04-01"
numTweets = 2500
numRuns = 6 # Call the function scraptweets numRuns = numTweets/300
scraptweets(search_words, date_since, numTweets, numRuns)

Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: si

Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tweet_mode
Unexpected parameter: since
Unexpected parameter: tw

no. of tweets scraped for run 1 is 2500
time take for 1 run to complete is 2.19 mins
